In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df_schedule = pd.read_csv("./data/season_prediction/schedule.csv", dtype={'GAME_ID': str})
#df_boxscores = pd.read_csv("./data/season_prediction/boxscores.csv", dtype={'GAME_ID': str})

In [6]:
from nba_api.stats.endpoints import playbyplayv2
import time
from tqdm import tqdm
def load_data_pbp(season_ids, limit=None):
    
    # load the data
    df_schedule = pd.read_csv("./data/season_prediction/schedule.csv", dtype={'GAME_ID': str})
    df_boxscores = pd.read_csv("./data/season_prediction/boxscores.csv", dtype={'GAME_ID': str})
    
    print(f'Get data for the season_ids: {season_ids}')
    
    #game_ids = ["0022001074", "0021900001"]
    game_ids = df_schedule[df_schedule['SEASON_ID'].isin(season_ids)]['GAME_ID'].unique()
    
    if limit is not None:
        game_ids = game_ids[0:limit]
    
    list_data = list()

    for game in tqdm(game_ids):
        
        try:
            # pbp call, append
            call = playbyplayv2.PlayByPlayV2(game_id=game, start_period=1, end_period=4)
            data_load = pd.concat(call.get_data_frames())
            list_data.append(data_load)
        
        except:
            print(f"Skipped the ID: {game}")
            next
        
        # sleep
        time.sleep(0.75)

    data_load = pd.concat(list_data)
    data_load = data_load[~data_load['GAME_ID'].isna()]
    
    return data_load

In [4]:
#data_load = load_data_pbp(season_ids=[22020, 22019, 22018])

  0%|          | 0/3369 [00:00<?, ?it/s]

Get data for the season_ids: [22020, 22019, 22018]


  0%|          | 1/3369 [00:02<1:52:18,  2.00s/it]

Skipped the ID: 0021800001


  0%|          | 2/3369 [00:03<1:46:44,  1.90s/it]


KeyboardInterrupt: 

In [ ]:
data = pd.read_csv("pbp_safe_data.csv")

In [ ]:
data.columns

In [ ]:
data = data_load

In [ ]:
df_schedule['SEASON_ID'].unique()

In [ ]:
data

In [7]:
def preprocessing_stint_data(data):
    
    # transform time, get game_time_left in seconds
    data['game_time_s'] = 60*data['PCTIMESTRING'].apply(lambda x: str(x).split(':')).apply(lambda x: x[0]).astype(float) + data['PCTIMESTRING'].apply(lambda x: str(x).split(':')).apply(lambda x: x[-1]).astype(float)
    data['game_time_left'] = (5 - data['PERIOD'])*data['game_time_s']
    data = data[~data['GAME_ID'].isna()]
    data = data[data['NEUTRALDESCRIPTION'].isna()]

    bool_ft = (data['EVENTMSGTYPE'] == 3).to_numpy() # Free-throws
    bool_fgm = (data['EVENTMSGTYPE'] == 1).to_numpy() # Field goals made
    bool_sub = (data['EVENTMSGTYPE'] == 8).to_numpy() # substitutions
    bool_tech = ((data['EVENTMSGTYPE'] == 6) & (data['EVENTMSGACTIONTYPE'].isin([10, 11, 16, 18, 25]))).to_numpy() # technical fouls
    bool_eject_tech = ((data['EVENTMSGTYPE'] == 11) & (data['EVENTMSGACTIONTYPE'] == 1)).to_numpy() # ejct 2nd technical fouls
    bool_ingame_plays = ~(bool_sub | bool_tech | bool_eject_tech)
    bool_away = (data['HOMEDESCRIPTION'].isna()).to_numpy()
    bool_home = (data['VISITORDESCRIPTION'].isna()).to_numpy()

    data.loc[bool_away, 'TEAM_LOCATION'] = "AWAY"
    data.loc[bool_home, 'TEAM_LOCATION'] = "HOME"
    
    return data, bool_ingame_plays

In [ ]:
data, bool_ingame_plays = preprocessing_stint_data(data)
id_vars=['GAME_ID','EVENTNUM', 'PERIOD', 'TEAM_LOCATION', 'PLAYER1_TEAM_ID']

In [8]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")
def create_stint(data):

    # create the stint_marker
    bool_subs = data['EVENTMSGTYPE'] == 8
    data.loc[bool_subs, 'stint_marker'] = np.arange(start=1, stop=np.sum(bool_subs) + 1)

    # create idx array
    inter_id = np.where(~data['stint_marker'].isna())[0] + 1
    last_id = data.shape[0]
    first_id = 0

    # idx
    idx = np.where(~data['stint_marker'].isna())[0] + 1

    list_stint = list()

    for i, ids in tqdm(enumerate(idx)):

        # first iteration
        if i == 0:
            tmp = data.iloc[:idx[0]]

        # in between
        else:
            tmp = data.iloc[idx[i - 1]:idx[i]]

        # append to list
        list_stint.append(tmp)

    # append the last iteration
    stint = data.iloc[idx[-1]:]
    stint['stint'] = i
    list_stint.append(stint)
    print(i)
    # insert the stint counter
    #for i, stint in enumerate(list_stint):
    #    stint['stint'] = i

    return pd.concat(list_stint)

In [9]:
def get_score(data):
    
    bool_score = ~data['SCORE'].isna()
    
    data.loc[bool_score, 'HOME_PTS'] = data[bool_score]['SCORE'].apply(lambda x: x.split(' - ')[0]).astype(int)
    data.loc[bool_score, 'AWAY_PTS'] = data[bool_score]['SCORE'].apply(lambda x: x.split(' - ')[1]).astype(int)
    
    data['HOME_PM'] = data['HOME_PTS'] - data['AWAY_PTS']
    data['AWAY_PM'] = data['AWAY_PTS'] - data['HOME_PTS']
    
    return data
def estimate_possessions(data, bool_ingame_plays):
    # get the estimated possessions - potential to be more accurate! Change forumula here
    tmp = np.floor((data[bool_ingame_plays].groupby(['GAME_ID', 'TEAM_LOCATION', 'stint']).count()/2))['EVENTNUM'].reset_index()
    tmp = tmp.rename(columns={'EVENTNUM':'EST_POSSESSIONS'})
    
    # merge to dataframe
    tmp2 = pd.merge(data, tmp, on=['GAME_ID', 'TEAM_LOCATION', 'stint'], how='left')
    bool_poss_na = tmp2['EST_POSSESSIONS'].isna()
    
    # set "empty stints" to zero
    tmp2.loc[bool_poss_na, 'EST_POSSESSIONS'] = None
    
    return tmp2
def estimate_pm_100(data):
    # formula for Plus Minus per 100 possessions
    data['HOME_PM_100'] = np.floor(data['HOME_PM'] * (100 / data['EST_POSSESSIONS']))
    data['AWAY_PM_100'] = np.floor(data['AWAY_PM'] * (100 / data['EST_POSSESSIONS']))
    
    # replace inf with nan
    data.loc[np.isinf(data['HOME_PM_100']), 'HOME_PM_100'] = np.nan
    data.loc[np.isinf(data['AWAY_PM_100']), 'AWAY_PM_100'] = np.nan
    
    return data

In [12]:
import time
from tqdm import tqdm
from nba_api.stats.endpoints import boxscoreadvancedv2
def get_roster_and_starters(data):
    
    list_starters = list()
    list_roster = list()
    
    print(data.head())
    
    unique_games = data['GAME_ID'].unique()
    
    print(unique_games)
    
    for gameid in tqdm(unique_games):
            print(gameid)
            
            call_boxscore = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=gameid)
            game = pd.concat(call_boxscore.get_data_frames())

            # get home or away
            print(game['TEAM_ID'].unique())
            away_team_id = game['TEAM_ID'].unique()[0] # maybe replace by table that has TEAM_LOCATION for all games

            game['TEAM_LOCATION'] = np.where(game['TEAM_ID'] == away_team_id, 'AWAY', 'HOME')


            tmp2 = game[~game['START_POSITION'].isna()][['START_POSITION', 'PLAYER_ID', 'GAME_ID', 'TEAM_ID', 'TEAM_LOCATION']]
            tmp2['STARTER'] = np.where((tmp2['START_POSITION'] == ""), False, True)
            tmp2 = tmp2.drop_duplicates()  # just in case

            list_starters.append(tmp2[tmp2['STARTER']])
            list_roster.append(tmp2)

            time.sleep(0.75)


            #print(f'ID {gameid} got skipped')  
            #next
    
    return pd.concat(list_starters), pd.concat(list_roster)

In [2]:
def get_all_subs(data):
    bool_sub = data['EVENTMSGTYPE'] == 8
    subs = data.loc[bool_sub, ['GAME_ID', 'stint', 'game_time_left', 'TEAM_LOCATION', 'PLAYER1_ID', 'PLAYER2_ID']]
    subs = subs.rename(columns={'PLAYER1_ID':'PLAYER_OUT_ID', 'PLAYER2_ID':'PLAYER_IN_ID'})
    return subs.sort_values(['GAME_ID', 'stint'])

In [3]:
def get_starting_lineup(starters):
    
    tmp = starters.groupby(['GAME_ID', 'TEAM_ID'])['PLAYER_ID'].unique().reset_index()
    starting_lineup = tmp.explode('PLAYER_ID')
    starting_lineup['stint'] = 0
    
    return starting_lineup

In [4]:
from tqdm import tqdm
def get_on_court(data, starters, sub):
    
    # init
    game_ids = data['GAME_ID'].unique()
    list_on_court = list()
    list_game_id = list()
    list_stint = list()
    
    # loop over all games
    for game_id in tqdm(game_ids):
        
        # form subsets
        game = data[data['GAME_ID'] == game_id]
        starting_lineup = starters[starters['GAME_ID'] == game_id]
        subs = sub[sub['GAME_ID'] == game_id]
        
        # prepare stints for the loop
        a = game['stint_marker'].unique()
        tmp = a[~np.isnan(a)]
        last_stint = np.max(tmp)
        stints = tmp - 1

        # get rosters for the game for both teams
        starters_home = starting_lineup[starting_lineup['TEAM_LOCATION'] == "HOME"]['PLAYER_ID'].unique()
        starters_away = starting_lineup[starting_lineup['TEAM_LOCATION'] == "AWAY"]['PLAYER_ID'].unique()
        
        # get the substitutions
        players_in, players_out = subs['PLAYER_IN_ID'].values, subs['PLAYER_OUT_ID'].values
        
        # more init
        on_court = np.zeros((stints.shape[0]+1, 10))
        lineup = np.append(starters_home, starters_away)
        
        # store game id, so lengths match
        list_game_id.append(game_id)

        for i, stint in enumerate(stints):
            
            j = i+1
            
            # for first iteration store starting lineup and first sub
            if stint == 0:
                on_court[i, :] = lineup
                lineup[lineup == players_out[i]] = players_in[i] # sub player out
                on_court[j, :] = lineup

            else:
                lineup[lineup == players_out[i]] = players_in[i]
                on_court[j, :] = lineup
            
            # store and keep track
            list_game_id.append(game_id)
            list_stint.append(stint)
        
        # store on court formations
        list_on_court.append(on_court)
        list_stint.append(last_stint)
        
        # numpy format
        a_on_court = np.concatenate(list_on_court)
        a_game_id = np.asarray(list_game_id)
        a_stint = np.asarray(list_stint)
        
        # create colnames
        col_names = ['stint']
        col_names.extend([f'HOME_{i}' for i in np.arange(start=1, stop=6)])
        col_names.extend([f'AWAY_{i}' for i in np.arange(start=1, stop=6)])
        col_names.extend(['GAME_ID'])
        
        # transform to dataframe
        df = pd.DataFrame(data=np.concatenate((a_stint.reshape(-1, 1), a_on_court, 
                                               a_game_id.reshape(-1, 1)), axis=1), columns=col_names)
        
        # adjust datatype
        df['stint'] = df['stint'].astype(float)
        
    return df

In [5]:
def merge_stint_pts(data, court_data, col_score):
    
    # store cols for merging and selecting
    col_merge = ['GAME_ID', 'stint']
    col_select = ['GAME_ID', 'stint', 'game_time_left']
    col_select.extend(col_score)

    # form subset
    data_subset = data[col_select]
    
    # merge
    df = pd.merge(court_data, data_subset, how='left', on=col_merge)
    
    # drop duplicates
    df = df.drop_duplicates()
    
    # impute values for missing scores
    tmp = df.groupby('GAME_ID')[col_score].ffill()
    df[col_score] = tmp
    
    # fill NA with zero - these are mostly before there was a score?
    df[col_score] = df[col_score].fillna(value=0)
    
    return df

In [3]:
def stints_to_dummy(data_stints):
    
    # colnames, create dummy out of player columns
    col_names = [f'HOME_{i}' for i in np.arange(start=1, stop=6)]
    col_names.extend([f'AWAY_{i}' for i in np.arange(start=1, stop=6)])
    data_dummy = pd.get_dummies(data_stints, prefix_sep='-', columns=col_names)
    
    # get series objects to track the players and their "position"
    series_location_player = pd.Series(data_dummy.columns[8:].values)
    series_position_player = series_location_player.apply(lambda x: x.split('-')[0])
    series_id_player = series_location_player.apply(lambda x: x.split('-')[1])
    series_id_player = series_id_player.apply(lambda x: x.replace('.0', ''))
    series_id_player = series_id_player.astype(int)
    
    return data_dummy, series_position_player, series_id_player

In [33]:
def estimate_model(data_dummy, ids_start, col_y, model):
    """
    :param ids_start: index of the column where ids start
    """
    #X = data_dummy.iloc[:, ids_start:].values
    #y = data_dummy[col_y].values
    
    print("Starting model fitting...")
    
    model.fit(data_dummy.iloc[:, ids_start:].values, data_dummy[col_y].values)
    
    player_ids = data_dummy.columns[ids_start:].values
    
    return model, player_ids

In [127]:
def show_scores_player(coef, series_id_player):
    
    # load player data
    player_data = pd.read_csv("players_data.csv")
    player_data['PLAYER_ID'] = player_data['id']

    # data array
    a = np.concatenate((player_ids.reshape(-1,1), 
                coef.reshape(-1,1)), axis=1)

    # create dataframe and merge
    df_tmp = pd.DataFrame(a, columns=['PLAYER_ID', 'SCORE'])
    df_tmp['PLAYER_ID'] = df_tmp['PLAYER_ID'].astype(str)
    df_tmp['PLAYER_ID'] = df_tmp['PLAYER_ID'].apply(lambda x: x.replace(".0", "")).astype(int)

    # merge the two
    df_result = pd.merge(df_tmp, player_data)
    
    return df_result.sort_values('SCORE', ascending=False)


In [ ]:
data.columns

In [ ]:
data.to_csv("pbp_safe_data.csv")

In [ ]:
data.shape

In [16]:
data = pd.read_csv("pbp_safe_data.csv", dtype={'GAME_ID':str})

In [17]:
bool_sub = (data['EVENTMSGTYPE'] == 8).to_numpy() # substitutions
bool_tech = ((data['EVENTMSGTYPE'] == 6) & (data['EVENTMSGACTIONTYPE'].isin([10, 11, 16, 18, 25]))).to_numpy() # technical fouls
bool_eject_tech = ((data['EVENTMSGTYPE'] == 11) & (data['EVENTMSGACTIONTYPE'] == 1)).to_numpy() # ejct 2nd technical fouls
bool_ingame_plays = ~(bool_sub | bool_tech | bool_eject_tech)

In [18]:
data = data.drop(['WCTIMESTRING', 'PCTIMESTRING', 'NEUTRALDESCRIPTION',
                  'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION', 'PLAYER3_TEAM_CITY',
                 'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER2_TEAM_CITY',
                 'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER1_TEAM_CITY',
                 'VIDEO_AVAILABLE_FLAG', 'SCOREMARGIN'], axis=1)

In [19]:
data

,Unnamed: 0,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,HOMEDESCRIPTION,VISITORDESCRIPTION,SCORE,PERSON1TYPE,...,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,game_time_s,game_time_left,TEAM_LOCATION
0,1,0021800002,4.0,10.0,0.0,1.0,Jump Ball Jones vs. Adams: Tip to Green,NaN,NaN,4.0,...,203500.0,Steven Adams,1.610613e+09,4.0,203110.0,Draymond Green,1.610613e+09,720.0,2880.0,HOME
1,2,0021800002,7.0,5.0,1.0,1.0,Thompson Bad Pass Turnover (P1.T1),Adams STEAL (1 STL),NaN,4.0,...,203500.0,Steven Adams,1.610613e+09,0.0,0.0,NaN,NaN,708.0,2832.0,NaN
2,3,0021800002,9.0,5.0,45.0,1.0,NaN,George Out of Bounds - Bad Pass Turnover Turno...,NaN,5.0,...,0.0,NaN,NaN,1.0,0.0,NaN,NaN,703.0,2812.0,AWAY
3,4,0021800002,10.0,1.0,80.0,1.0,Curry 24' 3PT Step Back Jump Shot (3 PTS) (Dur...,NaN,0 - 3,4.0,...,201142.0,Kevin Durant,1.610613e+09,0.0,0.0,NaN,NaN,691.0,2764.0,HOME
4,5,0021800002,12.0,2.0,78.0,1.0,NaN,MISS Adams 12' Floating Jump Shot,NaN,5.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,673.0,2692.0,AWAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536732,437,0022001072,632.0,3.0,12.0,4.0,NaN,Horton-Tucker Free Throw 2 of 2 (6 PTS),110 - 98,5.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,52.0,52.0,AWAY
1536733,438,0022001072,633.0,5.0,45.0,4.0,Marshall Out of Bounds - Bad Pass Turnover Tur...,NaN,NaN,4.0,...,0.0,NaN,NaN,1.0,0.0,NaN,NaN,35.0,35.0,HOME
1536734,439,0022001072,634.0,2.0,1.0,4.0,NaN,MISS Dudley 26' 3PT Jump Shot,NaN,5.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,28.0,28.0,AWAY
1536735,440,0022001072,635.0,4.0,0.0,4.0,Alexander-Walker REBOUND (Off:1 Def:3),NaN,NaN,4.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,23.0,23.0,HOME


In [ ]:
test = data.groupby(['GAME_ID']).apply(lambda x: create_stint(x))

In [ ]:
test = get_score(test)
test = estimate_possessions(data=test, bool_ingame_plays=bool_ingame_plays)
test = estimate_pm_100(data=test)

In [20]:
test = pd.read_csv("preprocessing_safe.csv", dtype={'GAME_ID': str})

In [21]:
test['GAME_ID'].unique()[0]

'0021800002'

In [20]:
call_boxscore = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=test['GAME_ID'].unique()[0])
game = pd.concat(call_boxscore.get_data_frames())

In [29]:
games_2020 = df_schedule[df_schedule['SEASON_ID'] == 22020]['GAME_ID'].unique()

In [31]:
test_sub = test[test['GAME_ID'].isin(games_2020)]

In [35]:
test_sub.shape

(482838, 34)

In [36]:
starters, roster = get_roster_and_starters(test_sub)

  0%|          | 0/1074 [00:00<?, ?it/s]

         Unnamed: 0  Unnamed: 0.1     GAME_ID  EVENTNUM  EVENTMSGTYPE  \
1053899     1053899             1  0022000001       4.0          10.0   
1053900     1053900             2  0022000001       7.0           5.0   
1053901     1053901             3  0022000001       8.0           6.0   
1053902     1053902             4  0022000001      10.0           3.0   
1053903     1053903             5  0022000001      11.0           3.0   

         EVENTMSGACTIONTYPE  PERIOD  \
1053899                 0.0     1.0   
1053900                45.0     1.0   
1053901                 2.0     1.0   
1053902                11.0     1.0   
1053903                12.0     1.0   

                                           HOMEDESCRIPTION  \
1053899                                                      
1053900  Jordan Out of Bounds - Bad Pass Turnover Turno...   
1053901                Irving S.FOUL (P1.T1) (S.Twardoski)   
1053902                                                NaN   
1053903         

  0%|          | 1/1074 [00:02<44:12,  2.47s/it]

0022000002
[1610612746 1610612747]


  0%|          | 2/1074 [00:04<39:53,  2.23s/it]

...

100%|█████████▉| 1072/1074 [30:52<00:02,  1.08s/it]

0022001074
[1610612746 1610612760]


100%|█████████▉| 1073/1074 [30:53<00:01,  1.07s/it]

0022001072
[1610612747 1610612740]


100%|██████████| 1074/1074 [30:56<00:00,  1.73s/it]


In [37]:
starters.to_csv("starters_safe_big.csv")
roster.to_csv("roster_safe_big.csv")

In [41]:
starting_lineups = get_starting_lineup(starters)

In [42]:
subs = get_all_subs(data=test)

In [43]:
subs = get_all_subs(data=test)
court = get_on_court(data=test_sub, starters=starters, sub=subs)
col_score = ['HOME_PTS', 'AWAY_PTS', 'HOME_PM', 'HOME_PM_100', 'EST_POSSESSIONS']
data_stints = merge_stint_pts(data=test_sub, court_data=court, col_score=col_score)

100%|██████████| 1074/1074 [03:52<00:00,  4.62it/s]


In [44]:
col_score = ['HOME_PTS', 'AWAY_PTS', 'HOME_PM', 'HOME_PM_100', 'EST_POSSESSIONS']
data_stints = merge_stint_pts(data=test_sub, court_data=court, col_score=col_score)

In [45]:
data_stints

,stint,HOME_1,HOME_2,HOME_3,HOME_4,HOME_5,AWAY_1,AWAY_2,AWAY_3,AWAY_4,AWAY_5,GAME_ID,game_time_left,HOME_PTS,AWAY_PTS,HOME_PM,HOME_PM_100,EST_POSSESSIONS
0,0.0,203925.0,201142.0,201599.0,203915.0,202681.0,203952.0,1629672.0,1630164.0,1626162.0,201939.0,0022000001,2880.0,0.0,0.0,0.0,0.0,13.0
1,0.0,203925.0,201142.0,201599.0,203915.0,202681.0,203952.0,1629672.0,1630164.0,1626162.0,201939.0,0022000001,2840.0,0.0,0.0,0.0,0.0,13.0
2,0.0,203925.0,201142.0,201599.0,203915.0,202681.0,203952.0,1629672.0,1630164.0,1626162.0,201939.0,0022000001,2792.0,0.0,0.0,0.0,0.0,13.0
3,0.0,203925.0,201142.0,201599.0,203915.0,202681.0,203952.0,1629672.0,1630164.0,1626162.0,201939.0,0022000001,2792.0,1.0,0.0,1.0,10.0,10.0
4,0.0,203925.0,201142.0,201599.0,203915.0,202681.0,203952.0,1629672.0,1630164.0,1626162.0,201939.0,0022000001,2792.0,2.0,0.0,2.0,20.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483907,38.0,1626195.0,1626195.0,1629638.0,1630184.0,1629712.0,203463.0,203076.0,203076.0,1628035.0,203463.0,0022001072,35.0,110.0,98.0,12.0,600.0,2.0
483908,38.0,1626195.0,1626195.0,1629638.0,1630184.0,1629712.0,203463.0,203076.0,203076.0,1628035.0,203463.0,0022001072,28.0,110.0,98.0,12.0,600.0,2.0
483909,38.0,1626195.0,1626195.0,1629638.0,1630184.0,1629712.0,203463.0,203076.0,203076.0,1628035.0,203463.0,0022001072,23.0,110.0,98.0,12.0,600.0,2.0
483910,38.0,1626195.0,1626195.0,1629638.0,1630184.0,1629712.0,203463.0,203076.0,203076.0,1628035.0,203463.0,0022001072,12.0,110.0,98.0,12.0,600.0,2.0


In [ ]:
def stints_to_dummy_unique(data_stints, col_scores):
    col_base = ['stint', 'GAME_ID', 'game_time_left']
    col_names = [f'HOME_{i}' for i in np.arange(start=1, stop=6)]
    col_names.extend([f'AWAY_{i}' for i in np.arange(start=1, stop=6)])
    
    data_slice = data_stints.drop(col_names, axis=1)
    
    # colnames, create dummy out of player columns
    col_names = [f'HOME_{i}' for i in np.arange(start=1, stop=6)]
    col_names.extend([f'AWAY_{i}' for i in np.arange(start=1, stop=6)])
    tmp = pd.melt(data_stints, id_vars=['stint', 'GAME_ID'], value_vars=col_names)
    tmp_dummy = pd.get_dummies(tmp, columns=['value'], prefix="", prefix_sep="")
    
    data_dummy = pd.merge(data_slice, tmp_dummy, on=['GAME_ID', 'stint'])
    
    data_dummy = data_dummy.drop('variable', axis=1)
    
    # get series objects to track the players and their "position"
    series_location_player = pd.Series(data_dummy.columns[8:].values)
    series_id_player = series_location_player.apply(lambda x: x.replace('.0', ''))
    series_id_player = series_id_player.astype(int)
    
    return data_dummy, series_id_player

In [ ]:
data_stints['GAME_ID'].unique().shape

In [ ]:
data_stints

In [47]:
def stints_to_dummy_unique_pm(data_stints):
    
    home_names = ['HOME_1', 'HOME_2', 'HOME_3', 'HOME_4', 'HOME_5']
    away_names = ['AWAY_1', 'AWAY_2', 'AWAY_3', 'AWAY_4', 'AWAY_5']

    data_home = data_stints.drop(away_names, axis=1)
    data_away = data_stints.drop(home_names, axis=1)

    home_long = pd.melt(data_home, id_vars=['stint', 'GAME_ID', 'HOME_PM_100', 'HOME_PM', 'HOME_PTS', 'AWAY_PTS'], 
                        value_vars=home_names)
    dummy_home = pd.get_dummies(home_long, columns=['value'], prefix="", prefix_sep="")

    away_long = pd.melt(data_away, id_vars=['stint', 'GAME_ID', 'HOME_PM_100', 'HOME_PM', 'HOME_PTS', 'AWAY_PTS'],
                        value_vars=away_names)
    dummy_away = pd.get_dummies(away_long, columns=['value'], prefix="", prefix_sep="")
    dummy_away.iloc[:, 3:] =  dummy_away.iloc[:, 3:] * (-1)

    dummy_pm = pd.concat([dummy_home, dummy_away], join='inner')
    dummy_pm = dummy_pm.drop('variable', axis=1)
    
    # get series objects to track the players and their "position"
    series_location_player = pd.Series(dummy_pm.columns[6:].values)
    series_id_player = series_location_player.apply(lambda x: x.replace('.0', ''))
    series_id_player = series_id_player.astype(int)

    return dummy_pm, series_id_player

In [48]:
import pandas as pd

In [49]:
dummy_pm, player_ids = stints_to_dummy_unique_pm(data_stints=data_stints)

In [50]:
player_ids.unique().shape

(529,)

In [51]:
dummy_pm.shape

(4364530, 535)

**How to deal with empty stints? Leave them in or out? Do an ffill? How to deal with NA Home PTS?**

In [53]:
dummy_pm.to_csv("dummy_safe_long.csv") 
player_ids.to_csv("player_ids_model.csv")

In [5]:
#unique_games = df_schedule[df_schedule['SEASON_ID'] == 22020]['GAME_ID'].unique()
tmp = pd.read_csv("game_id_2020.csv", dtype={'GAME_ID':str})
unique_games = tmp['GAME_ID'].unique()

In [6]:
sample_games = np.random.choice(unique_games, size=500, replace=False)

In [8]:
cols_to_drop = ['stint', 'HOME_PTS', 'AWAY_PTS', 'HOME_PM_100', 'Unnamed: 0']
cols = list(pd.read_csv("dummy_safe_long.csv", nrows = 1))
data_dummy = pd.read_csv("dummy_safe_long.csv", dtype={'GAME_ID':str}, usecols =[i for i in cols if ~(i in cols_to_drop)])

In [9]:
data_sample = data_dummy[data_dummy['GAME_ID'].isin(sample_games)]

In [10]:
data_dummy = ''

In [11]:
data_sample = data_sample.drop(cols_to_drop, axis=1)

In [12]:
data_sample.shape

(2027930, 531)

In [ ]:
from sklearn.linear_model import RidgeCV
model = RidgeCV(alphas=[1e-3, 1e-1, 1e1, 1e3, 1e4], normalize=True, cv=3)

In [13]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1.5e3) #1e-3

In [14]:
model, player_ids = estimate_model(data_dummy=data_sample, ids_start=2, col_y="HOME_PM", model=model)

Starting model fitting...


In [31]:
player_ids = data_sample.columns[2:].values

In [ ]:
# model.best_score_, model.alpha_

In [ ]:
model.coef_.shape

In [ ]:
#coef = model.coef_[:-1]
#coef = coef[:-1]

In [22]:
coef = model.coef_

In [25]:
coef.shape

(529,)

In [35]:
model.coef_.shape


(529,)

In [34]:
player_ids.shape


(529,)

In [15]:
import pickle
pickle.dump(model, open("model_ridge_no_cv", 'wb'))

Merge daten müssen zur Saison passen

In [128]:
scores = show_scores_player(coef=coef, series_id_player=player_ids)
scores.to_csv("player_scores.csv")

In [129]:
scores

,PLAYER_ID,SCORE,id,player_names,position,team
158,1629620,8.810845,1629620,Justin Robinson,G,OKC
371,203504,6.595839,203504,Trey Burke,G,DAL
45,1627780,6.206122,1627780,Gary Payton II,G,GSW
178,1629645,5.082931,1629645,Kevin Porter,G,HOU
90,1628467,4.385644,1628467,Maxi Kleber,F,DAL
...,...,...,...,...,...,...
92,1628539,-4.36008,1628539,Mychal Mulder,G,GSW
286,201580,-4.729927,201580,JaVale McGee,C,DEN
396,203953,-4.916955,203953,Jabari Parker,F,BOS
300,201961,-5.089548,201961,Wayne Ellington,G,DET


In [130]:
scores.to_csv("500_2020_player_scores.csv")


In [3]:
scores = pd.read_csv("500_2020_player_scores.csv")

In [53]:
from src.get_data import get_clean_player_data
import time
from tqdm import tqdm
list_player = list()

for player in tqdm(scores['PLAYER_ID']):
    list_player.append(get_clean_player_data(player_id=player))
    time.sleep(0.75)
    
data_player = pd.concat(list_player)

100%|██████████| 529/529 [20:57<00:00,  2.38s/it]


In [88]:
data_player['PLAYER_ID'] = data_player['PLAYER_ID'].astype(int)

In [89]:
data_player.to_csv("player_data_for_scores.csv")

In [105]:
last_season = data_player[data_player['SEASON_ID'] == "2020-21"]


In [131]:
full_player_rating = pd.merge(last_season, scores, on='PLAYER_ID', how='right')
full_player_rating.sort_values('SCORE', ascending=False)
#full_player_rating[~full_player_rating['id'].isna()].sort_values('SCORE', ascending=False).head(30)

,PLAYER_ID,SEASON_ID,TEAM_ID,PLAYER_AGE,GP,GS,MIN,FG_PCT,FGA,FG3_PCT,...,STL,BLK,TOV,PF,PTS,SCORE,id,player_names,position,team
0,1629620,2020-21,1.610613e+09,23.0,9,0,9.8,0.333,2.3,0.286,...,0.3,0.0,0.2,1.1,2.3,8.810845,1629620,Justin Robinson,G,OKC
1,203504,2020-21,1.610613e+09,28.0,62,1,14.7,0.428,5.7,0.354,...,0.6,0.1,0.5,1.0,6.6,6.595839,203504,Trey Burke,G,DAL
2,1627780,2020-21,1.610613e+09,28.0,10,0,4.0,0.769,1.3,0.5,...,0.6,0.1,0.1,0.8,2.5,6.206122,1627780,Gary Payton II,G,GSW
3,1629645,2020-21,1.610613e+09,21.0,26,23,32.1,0.425,14.1,0.311,...,0.7,0.3,3.5,2.3,16.6,5.082931,1629645,Kevin Porter,G,HOU
4,1628467,2020-21,1.610613e+09,29.0,50,40,26.8,0.422,5.5,0.41,...,0.5,0.7,0.6,2.1,7.1,4.385644,1628467,Maxi Kleber,F,DAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,201580,2020-21,1.610613e+09,33.0,13,1,13.5,0.478,5.2,0.0,...,0.2,1.1,1.2,3.2,5.5,-4.729927,201580,JaVale McGee,C,DEN
478,203953,2020-21,1.610613e+09,26.0,3,0,9.1,0.571,2.3,0.0,...,0.0,0.3,1.0,1.3,2.7,-4.916955,203953,Jabari Parker,F,BOS
479,203953,2020-21,1.610613e+09,26.0,10,0,13.8,0.542,4.8,0.2,...,0.1,0.4,0.7,0.9,6.4,-4.916955,203953,Jabari Parker,F,BOS
480,201961,2020-21,1.610613e+09,33.0,46,31,22.0,0.441,7.3,0.422,...,0.4,0.2,0.7,1.3,9.6,-5.089548,201961,Wayne Ellington,G,DET


In [132]:
full_player_rating.to_csv("full_player_rating.csv")

In [133]:
scores.sort_values(['SCORE'], ascending=False)

,PLAYER_ID,SCORE,id,player_names,position,team
158,1629620,8.810845,1629620,Justin Robinson,G,OKC
371,203504,6.595839,203504,Trey Burke,G,DAL
45,1627780,6.206122,1627780,Gary Payton II,G,GSW
178,1629645,5.082931,1629645,Kevin Porter,G,HOU
90,1628467,4.385644,1628467,Maxi Kleber,F,DAL
...,...,...,...,...,...,...
92,1628539,-4.36008,1628539,Mychal Mulder,G,GSW
286,201580,-4.729927,201580,JaVale McGee,C,DEN
396,203953,-4.916955,203953,Jabari Parker,F,BOS
300,201961,-5.089548,201961,Wayne Ellington,G,DET


In [138]:
scores

,PLAYER_ID,SCORE,id,player_names,position,team
158,1629620,8.810845,1629620,Justin Robinson,G,OKC
371,203504,6.595839,203504,Trey Burke,G,DAL
45,1627780,6.206122,1627780,Gary Payton II,G,GSW
178,1629645,5.082931,1629645,Kevin Porter,G,HOU
90,1628467,4.385644,1628467,Maxi Kleber,F,DAL
...,...,...,...,...,...,...
92,1628539,-4.36008,1628539,Mychal Mulder,G,GSW
286,201580,-4.729927,201580,JaVale McGee,C,DEN
396,203953,-4.916955,203953,Jabari Parker,F,BOS
300,201961,-5.089548,201961,Wayne Ellington,G,DET
